In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 6.6 MB 47.8 MB/s 
     |████████████████████████████████| 895 kB 72.3 MB/s 
     |████████████████████████████████| 77 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 75.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install hanja

     |████████████████████████████████| 120 kB 4.3 MB/s 
     |████████████████████████████████| 265 kB 30.3 MB/s 
     |████████████████████████████████| 297 kB 50.8 MB/s 
     |████████████████████████████████| 207 kB 70.3 MB/s 
     |████████████████████████████████| 207 kB 74.2 MB/s 
     |████████████████████████████████| 207 kB 70.1 MB/s 
     |████████████████████████████████| 213 kB 75.6 MB/s 
     |████████████████████████████████| 213 kB 68.5 MB/s 
     |████████████████████████████████| 213 kB 73.4 MB/s 
     |████████████████████████████████| 213 kB 55.6 MB/s 
     |████████████████████████████████| 253 kB 62.3 MB/s 
     |████████████████████████████████| 252 kB 67.2 MB/s 
     |████████████████████████████████| 252 kB 68.7 MB/s 
     |████████████████████████████████| 242 kB 67.5 MB/s 
     |████████████████████████████████| 242 kB 65.9 MB/s 
     |████████████████████████████████| 242 kB 61.0 MB/s 
     |████████████████████████████████| 229 kB 66.6 MB/s 
     |█████████

In [5]:
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import hanja
from imblearn.over_sampling import RandomOverSampler
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from transformers import AdamW
from collections import defaultdict
import torchvision.models as models
import os

In [6]:
from google.colab import drive
drive.mount('/content/drive')
os.listdir("/content/drive/My Drive/3조")

Mounted at /content/drive


['1.기획안&WBS',
 '2.수행일지',
 '3.최종포트폴리오',
 '4.과제',
 '멘토링',
 '회의록',
 '[D11&12]기업요구사항 기반의 문제해결 프로젝트.pdf',
 'Data',
 '빅데이터분석프로젝트_단계별공정.pdf',
 'DS_folder',
 'DE']

In [7]:
PATH = '/content/drive/My Drive/3조/DS_folder/jupyter notebook/Data/'

In [8]:
train = pd.read_csv(PATH + 'data.csv', index_col=0)
test = pd.read_csv(PATH + 'val.csv', index_col=0)

In [9]:
train

,target,data
0,0,지금 난 기분이 너무 좋아.
1,0,나도 조카가 생겨! 너무 기뻐.
2,1,거래처와의 다음 계약이 무산될까봐 불안해.
3,2,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...
4,2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.
...,...,...
199367,2,일단 좀 쉬어야 할 것 같아. 남자 친구와의 추억도 정리하고 마음도 정리해야겠어.
199368,2,나는 아들이 엄청나게 좋아할 거라 기대했는데 실망스러워.
199369,5,먼저 업무 인계서부터 천천히 작성 해야지.
199370,0,기분이 좋아. 나도 일 때문에 바쁠 때는 미리 얘기하기로 했어.


In [10]:
x_train = train['data']
y_train = train['target']
x_test = test['data']
y_test = test['target']

---

In [11]:
tokenizer_bert_kor_base = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

In [13]:
titles_t = x_train.to_numpy().reshape(-1,1)
labels_t = y_train.to_numpy().reshape(-1,1)
# oversample = RandomOverSampler()
# X_over, y_over = oversample.fit_resample(titles_t, labels_t)
# train = pd.DataFrame({'title':X_over.reshape(-1), 'topic_idx':y_over.reshape(-1)})
train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

In [14]:
titles_t = x_test.to_numpy().reshape(-1,1)
labels_t = y_test.to_numpy().reshape(-1,1)
# oversample = RandomOverSampler()
# X_over, y_over = oversample.fit_resample(titles_t, labels_t)
# train = pd.DataFrame({'title':X_over.reshape(-1), 'topic_idx':y_over.reshape(-1)})
test = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

In [16]:
# 뉴스 데이터 클래스 생성
class NewsSubjectDataset(Dataset):
  def __init__(self, subjects, targets, tokenizer, max_len):
    self.subjects = subjects
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.subjects)
  def __getitem__(self, item):
    subject = str(self.subjects[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      subject,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding = 'max_length',
      truncation = True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'subject_text': subject,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }
def create_data_loader(df, tokenizer, max_len, batch_size, shuffle_=False, valid=False):
  if valid:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=np.zeros(len(df)),
      tokenizer=tokenizer,
      max_len=max_len
      )
  else:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=df.topic_idx.to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len
    )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4,
    shuffle = shuffle_
  )
  
# 데이터 로더 생성
BATCH_SIZE =64
MAX_LEN =32
train_data_loader = create_data_loader(train, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, shuffle_=True)
valid_data_loader = create_data_loader(test, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, valid=True)
class NewsSubjectClassifier(nn.Module):
  def __init__(self, n_classes):
    super(NewsSubjectClassifier, self).__init__()
    self.bert = BertModel.from_pretrained("kykim/bert-kor-base")
    self.drop = nn.Dropout(p=0.5)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
       return_dict=False
    )
    output = self.drop(pooled_output)
    return self.out(output)

device = torch.device("cpu")

def get_predictions(model, data_loader):
  model = model.eval()
  subject_texts = []
  predictions = []
  prediction_probs = []
  with torch.no_grad():
    for d in data_loader:
      texts = d["subject_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      subject_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  return subject_texts, predictions, prediction_probs

import gc

EPOCHS = 2

In [17]:
model_bert_kor_base = NewsSubjectClassifier(n_classes=6).to(device)
optimizer = AdamW(model_bert_kor_base.parameters(), lr=1e-4)
total_steps = len(train_data_loader) * EPOCHS
## cosine_scheduler or linear,  warmup or no warmup
scheduler = get_cosine_schedule_with_warmup(
  optimizer,
  num_warmup_steps=int(total_steps*0.1),
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

from tqdm import tqdm
def train_epoch(model,data_loader,loss_fn,optimizer,device,scheduler,n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in tqdm(data_loader):
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_examples, np.mean(losses)

Downloading:   0%|          | 0.00/454M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(
    model_bert_kor_base,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')

y_review_texts, y_pred, y_pred_probs = get_predictions(
  model_bert_kor_base,
  valid_data_loader
)
answers = []
answers.append(y_pred.tolist())
gc.collect()
torch.cuda.empty_cache()

Epoch 1/2
----------


100%|██████████| 3116/3116 [4:57:01<00:00,  5.72s/it]


Train loss 1.3734158879296618 accuracy 0.4649298798226431
Epoch 2/2
----------


 31%|███       | 962/3116 [1:32:46<3:25:52,  5.73s/it]

In [ ]:
torch.save(model_bert_kor_base, PATH+'model_albert_kor_base.pth')

In [ ]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model_bert_kor_base.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss,
            }, './모델_bert.tar')

In [ ]:
model = torch.load(PATH+"model_bert_kor_base.pth")

In [ ]:
state_dict = model.load_state_dict(torch.load(PATH + 'model_state.pth'))

In [ ]:
model.eval()

In [ ]:
subject_texts, predictions, prediction_probs = get_predictions(model, valid_data_loader)

In [ ]:
df = pd.DataFrame({'data':subject_texts, 'predict':predictions, 'target':y_test})
df

In [ ]:
# 정확도
df[df['predict'] == df['target']].shape[0] / df.shape[0]

In [ ]:
df['data']

In [ ]:
# def change(x):
#     if x == '기쁨':
#         return 0
#     elif x == '불안':
#         return 1
#     elif x == '슬픔':
#         return 2
#     elif x ==‘당황:
#         return 3
#     elif x == ‘상처’:
#         return 4
#     elif x == ‘분노’:
#         return 5